In [7]:
using Plots
using DifferentialEquations

# Система дифференциальных уравнений для конкуренции двух видов
function competition_system!(du, u, p, t)
    e1, e2 = p
    u1, u2 = u
    du[1] = u1 * (1 - u1 - e1 * u2)  # Вид 1
    du[2] = u2 * (1 - u2 - e2 * u1)  # Вид 2
end

# Функция для создания фазового портрета
function create_phase_portrait(e1, e2, scenario_name, filename)
    # Создаем график
    plt = plot(size=(800, 600), legend=:topright)

    # Различные начальные условия
    initial_conditions = [
        [0.1, 0.8], [0.1, 0.5], [0.1, 0.2],
        [0.3, 0.8], [0.3, 0.5], [0.3, 0.2],
        [0.6, 0.8], [0.6, 0.5], [0.6, 0.2],
        [0.8, 0.8], [0.8, 0.5], [0.8, 0.1]
    ]

    colors = palette(:viridis, length(initial_conditions))

    # Рисуем траектории для разных начальных условий
    for (i, u0) in enumerate(initial_conditions)
        p = (e1, e2)
        prob = ODEProblem(competition_system!, u0, (0.0, 20.0), p)
        sol = solve(prob, Tsit5())

        u1_traj = [point[1] for point in sol.u]
        u2_traj = [point[2] for point in sol.u]

        plot!(u1_traj, u2_traj,
              linewidth=2,
              linecolor=colors[i],
              label=false,
              alpha=0.7)

        # Начальная точка
        scatter!([u0[1]], [u0[2]],
                markersize=4,
                markercolor=colors[i],
                markerstrokewidth=0,
                label=false)
    end

    # Рисуем изоклины
    u1_range = 0:0.01:1.2
    u2_iso1 = @. (1 - u1_range) / e1  # Изоклина для вида 1: du1/dt = 0
    u1_iso2 = @. (1 - u1_range) / e2  # Изоклина для вида 2: du2/dt = 0

    plot!(u1_range, u2_iso1,
          linewidth=3,
          linestyle=:dash,
          linecolor=:blue,
          label="Изоклина вида 1",
          alpha=0.8)

    plot!(u1_iso2, u1_range,
          linewidth=3,
          linestyle=:dash,
          linecolor=:red,
          label="Изоклина вида 2",
          alpha=0.8)

    # Настройка внешнего вида
    plot!(xlims=(0, 1.1), ylims=(0, 1.1),
          xlabel="Численность вида 1 (u₁)",
          ylabel="Численность вида 2 (u₂)",
          title="Фазовый портрет системы конкуренции\n$scenario_name (e₁ = $e1, e₂ = $e2)",
          titlefontsize=12,
          grid=true, gridstyle=:dot, gridalpha=0.3)

    # Сохраняем график
    savefig(plt, filename)
    return plt
end

# Создаем фазовые портреты для трех сценариев
println("Создание фазовых портретов...")

# Сценарий 1: Устойчивое сосуществование
p1 = create_phase_portrait(0.3, 0.4, "Устойчивое сосуществование", "phase_portrait_coexistence.png")

# Сценарий 2: Победа вида 1
p2 = create_phase_portrait(0.5, 1.2, "Победа вида 1", "phase_portrait_species1_wins.png")

# Сценарий 3: Победа вида 2
p3 = create_phase_portrait(1.2, 0.5, "Победа вида 2", "phase_portrait_species2_wins.png")

println("Готово! Созданы файлы:")
println("✓ phase_portrait_coexistence.png")
println("✓ phase_portrait_species1_wins.png")
println("✓ phase_portrait_species2_wins.png")

# Дополнительная функция для создания графика с временными рядами
function create_time_series_plot(e1, e2, scenario_name, filename)
    u0 = [0.3, 0.7]
    p = (e1, e2)
    prob = ODEProblem(competition_system!, u0, (0.0, 20.0), p)
    sol = solve(prob, Tsit5())

    t = sol.t
    u1 = [point[1] for point in sol.u]
    u2 = [point[2] for point in sol.u]

    plt = plot(t, u1,
               linewidth=3,
               label="Вид 1 (u₁)",
               color=:blue,
               xlabel="Время",
               ylabel="Численность",
               title="Динамика численностей: $scenario_name",
               grid=true)

    plot!(t, u2,
          linewidth=3,
          label="Вид 2 (u₂)",
          color=:red)

    savefig(plt, filename)
    return plt
end

# Создаем графики временных рядов
println("\nСоздание графиков временных рядов...")

ts1 = create_time_series_plot(0.3, 0.4, "Устойчивое сосуществование", "time_series_coexistence.png")
ts2 = create_time_series_plot(0.5, 1.2, "Победа вида 1", "time_series_species1_wins.png")
ts3 = create_time_series_plot(1.2, 0.5, "Победа вида 2", "time_series_species2_wins.png")

println("✓ time_series_coexistence.png")
println("✓ time_series_species1_wins.png")
println("✓ time_series_species2_wins.png")

Создание фазовых портретов...
Готово! Созданы файлы:
✓ phase_portrait_coexistence.png
✓ phase_portrait_species1_wins.png
✓ phase_portrait_species2_wins.png

Создание графиков временных рядов...
✓ time_series_coexistence.png
✓ time_series_species1_wins.png
✓ time_series_species2_wins.png
